<a href="https://colab.research.google.com/github/itssofiarce/NLP-2025/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Procesamiento del Lenguaje Natural - 2025

### Trabajo Práctico

Arce, Sofía.

## Preparación de entorno

In [1]:
# Clonamos el reposotorio y el directorio de los datos
import os

REPO_NAME = "NLP-2025"
if REPO_NAME not in os.getcwd():
  if not os.path.exists(REPO_NAME):
    !git clone https://github.com/itssofiarce/{REPO_NAME}.git
  os.chdir(REPO_NAME)

### Pre analísis de la información .txt

In [2]:
# Texto mas extenso
longitud = {}
informacion = os.listdir('/content/NLP-2025/NLP_CASCADIA/datos/información')

for texto in informacion:
  contenido = open(f'/content/NLP-2025/NLP_CASCADIA/datos/información/{texto}', 'r')
  # Cuento solo las palabras

  longitud[texto] = len(contenido.read().split())

print(longitud)


{'whatboardgame_review.txt': 2288, 'board_game_co_uk_guide.txt': 127, 'flatout_games.txt': 298, 'boardgamereview_review.txt': 2606, 'cascadia_manual.txt': 6734, 'bluehighwaygames_description.txt': 908, 'oneboardfamily_review.txt': 1592}


En base al output de arriba el texto mas extenso es el manual de cascadia con 6734 palabras.



Para facilitar el desarrollo del Trabajo Práctico crearé la clase "TextoParaAnalizar" con los metodós necesarios para completar los enunciados.

In [3]:
import pandas as pd

In [4]:
pip install numpy

In [10]:
# Funciones adicionales
def entorno_modelo(tipo_embedding):
  if tipo_embedding == "CountVectorizer":

    from sklearn.feature_extraction.text import CountVectorizer
    !pip install numpy
    model = CountVectorizer()


  elif tipo_embedding == "WORD2VEC":
    !pip install gdown
    #!pip install numpy==1.23.5 gensim==4.3.2 scipy==1.10.1
    !pip install gensim


    from gensim.models import Word2Vec  # Change import to include models
    model = Word2Vec(window=5, min_count=2, workers=4, sg=0)

  elif tipo_embedding == "sentencetransformer":
    !pip install -U sentence-transformers
    from sentence_transformers import SentenceTransformer
    model = SentenceTransformer('all-MiniLM-L6-v2')

  return model


In [6]:
class TextoParaAnalizar:

  def __init__(self, text):
    # Validar path del archivo
    if not os.path.exists(text):
      raise Exception('El archivo no existe')
    self.text = text

  def __str__(self):
    """ Devuelve el contenido del texto """
    with open(self.text, 'r') as f:
      return f.read()

  def fragmentos(self, fragmento="palabra"):
    """ Devuelve una lista de fragmentos del texto.
      - "Fragmento: oracion"
      - "Fragmento: palabra"
    """
    import re
    texto_limpio = re.sub(r'[().!]', '', self.__str__())
    if fragmento == "oracion":
      texto_limpio =[line for line in texto_limpio.splitlines() if line.strip()]
      return texto_limpio
    return texto_limpio.split()


  def vectorizar(self, tipo: str = "WORD2VEC"):
    """ Vectoriza los fragmentos y los devuelve en un dataset. Por default usa ul modelo ContextDependent como word2vec.
        tipo:
        - "ContextDependent: CountVectorizer"
        - "ContextIndependent: "WORD2VEC" """
    import pandas as pd
    modelo = entorno_modelo(tipo)
    oraciones = self.fragmentos("oracion")
    embeddings = []

    if tipo == "CountVectorizer":
      X = modelo.fit_transform(oraciones)
      response_new = pd.DataFrame(X.toarray(), columns=modelo.get_feature_names_out())

    elif tipo == "WORD2VEC":

      data = pd.DataFrame(oraciones, columns=['oracion'])
      from gensim import utils
      response_new = data.oracion.apply(utils.simple_preprocess)

    elif tipo == "sentencetransformer":
      response_new = modelo.encode(oraciones)

    return response_new, modelo

## EJERCICIO 2
Apoyándose en la sección de información. Separa en fragmentos un texto extenso extraído y vectoriza cada fragmento con alguno de los modelos de embedding vistos en clases.

In [7]:
texto_extenso = TextoParaAnalizar('/content/NLP-2025/NLP_CASCADIA/datos/información/cascadia_manual.txt')

In [11]:
# Separo en palabras y vectorizo
dataframe, model = texto_extenso.vectorizar("WORD2VEC")

# Train the model
model.build_vocab(dataframe, progress_per=1000)
model.train(dataframe, total_examples=model.corpus_count, epochs=model.epochs)

# Save the trained model
model.save("./responses.model")

En el bloque de codigo de arriba, dataframe es un diccionario que almacena en forma de lista, las palabras de las oraciones, esto el permite a word2vec aprender mejor el contexto de las mismas. Luego entrenamos el modelo y lo guardamos.

In [12]:
# Palabra de prueba
model.wv["rules"]

array([-0.00322331,  0.00729358,  0.00079845,  0.00532187,  0.00581209,
       -0.0254052 ,  0.00618105,  0.02501567, -0.00389221, -0.01079057,
        0.00054122, -0.01767658, -0.00696769,  0.01310214, -0.00721239,
        0.00803325,  0.00382074, -0.01448411, -0.00078555, -0.01801282,
        0.0071508 ,  0.01672942,  0.00285084, -0.0124024 , -0.01202836,
        0.01040764, -0.00226401,  0.00447703, -0.00239419,  0.01037169,
        0.01016498,  0.00523393,  0.00278911, -0.01365996, -0.01422835,
        0.0098418 , -0.00269039, -0.00534501, -0.00334259, -0.01817841,
        0.01117781, -0.01899633, -0.01230346, -0.01187329,  0.01918026,
        0.00672027, -0.0008671 , -0.01227087,  0.00173522, -0.0020884 ,
        0.00171264, -0.0002115 , -0.004914  ,  0.00504383, -0.00945198,
        0.00904872, -0.00123475, -0.00643598, -0.0176939 ,  0.00445501,
       -0.00467835,  0.00352812,  0.00096327, -0.01280226, -0.0014025 ,
        0.016837  ,  0.01577605,  0.00917757, -0.00538982,  0.01

In [13]:
model.wv.similarity(w1="rules", w2="games")

0.8029425

Las palabras "rules" y "games" tienen una similitud cerca de uno por lo que significa que son palabras cuyos embeddings estan cerca en el espacio y por lo tanto, estan relacionadas en cuanto a significado. Ahora entrenaré otro modelo para probarlo con frases.

In [14]:
# Separo en palabras y vectorizo
dataframe, model = texto_extenso.vectorizar("CountVectorizer")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [8]:
# Separo en frases
dataframe, model = texto_extenso.vectorizar("sentencetransformer")


RuntimeError: Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
module 'numpy' has no attribute 'dtypes'

Luego realiza un análisis de similitud de texto ingresando varias frases a buscar semánticamente, compare distintas técnicas de distancias vistas en clases, elija la mejor y justifique la razón por la que esa técnica se ajusta para este tipo de búsquedas.

OPCIONAL: Visualizar en 3D aplicando PCA o t-SNE la ubicación de los fragmentos y la query ingresada vectorizada en el espacio. Realizar una observación sobre la visualización.


In [ ]:
# buscar frases semanticamente en el dataframe
from sklearn.metrics.pairwise import cosine_similarity



de la teoria:: Los métodos Count Vectorizer y TF-IDF capturan la aparición o frecuencia de palabras. Si bien admiten el uso
de similitud de coseno, la cercanía estará dada por las similitudes en cuanto a la presencia o cuenta de las
mismas palabras, y no por su similitud semántica o de contexto, como en el caso de los modelos de
embeddings.


## EJERCICIO 3
Apoyándose nuevamente en la sección de información. Recoge un texto extenso extraído, divídelos en fragmentos, luego realiza extracciones de sustantivos (POS) y categoriza estos sustantivos (NER), a continuación realiza una búsqueda de similitud filtrando por sustantivos, compara las distintas técnicas de distancias vistas en clases, elija la mejor y justifique la razón por la que esta técnica se ajusta para este tipo de búsquedas.


## EJERCICIO 4
Mediante detección de idioma, separar los archivos en distintos lenguajes y guardar esa
información en un dataframe.


##EJERCICIO 5
En el caso de las reseñas realizadas por usuarios, utiliza análisis de sentimientos con modelos pre entrenados y guarda la clasificación predecida de cada reseña.

Luego, crea un sistema de búsquedas por similitud semántica y que permita filtrar por sentimiento para obtener.


## EJERCICIO 6
Crea un set de datos de consultas (más de 300 preguntas en total) y categorizalas entre la
fuente de datos que pueda llegar a responder esa pregunta entre estadísticas,
información y relaciones.
Por ejemplo:
- ¿Cómo gano en el ajedrez? -> Información
- ¿Quién trabajó para el ta-te-ti? -> Relaciones
- ¿Qué puntaje tienen las damas? -> Estadística
A continuación, transforma esas consultas en vectores y entrena un modelo de clasificación
(a gusto del estudiante) en donde pueda predecir la categoría a través de la consulta
ingresada.
Agregar métricas y análisis durante todo el desarrollo, trabaje en varios modelos y
comparelos.